# Detecção de fraude em cartões de crédito

## Descrição
<p></p>
<div style="text-align: justify"> Os conjuntos de dados contêm transações feitas por cartões de crédito em setembro de 2013 por titulares de cartões europeus.
Este conjunto de dados apresenta as transações que ocorreram no período de dois dias, no qual temos 492 fraudes em 284.807 transações. O conjunto de dados é altamente desequilibrado, a classe positiva (fraudes) corresponde a 0,172% de todas as transações. </div>

<div style="text-align: justify">Ele contém apenas variáveis de entrada numéricas que são o resultado de uma transformação PCA. Infelizmente, devido a questões de confidencialidade, não podemos fornecer características originais e mais informações de fundo sobre os dados. As características V1, V2, etc. são os principais componentes obtidos com PCA, as únicas características que não foram transformadas com PCA são "Time" e "Amount". A variável "Tempo" contém os segundos transcorridos entre cada transação e a primeira transação no conjunto de dados. A variável "Amount" é a transação "Montante". A variável "Class" é a variável de resposta e toma valor 1 em caso de fraude e 0 em caso contrário. </div>
<p>
   <b>Dataset</b>: <a href="https://www.kaggle.com/mlg-ulb/creditcardfraud">https://www.kaggle.com/mlg-ulb/creditcardfraud</a>


## 1. Pacotes

In [ ]:
# Importando pacotes
library(tidyverse)
library(caTools)
library(caret)
library(pROC)
library(precrec)
library(ROCR)
library(modEvA)
library(rpart)
library(rpart.plot)
library(scales)
library(e1071)
library(randomForest)
library(descr)


## 2. Importar dados

In [ ]:
dados<- read.csv("creditcard.csv")

In [ ]:
# Visualização das 5 primeiras linhas
head(dados)

In [ ]:
# Visualização das 5 últimas linhas
tail(dados)

In [ ]:
# Sumário dos dados
summary(dados)

## 3. Preparar dados


<i> Após ter uma visão geral do dataset com as funções head() e tail(), devemos checar a existência de valores nulos (NAs) para uma possível correção, e transformar a variável "Class" em factor.

In [ ]:
# Checar NA
f=function(x){any(is.na(x))}
check.na=apply(dados,2,f);
check.na

In [ ]:
# Variável Class como factor
dados$Class<- as.factor(dados$Class)

## Explorar os dados

In [ ]:
# Visualização resumida de colunas e dados
glimpse(dados)

<i> <div style="text-align: justify"> Como podemos ver abaixo, há grande discrepância entre o número de transações legais e de transações fraudulentas na contagem dos valores da variável "Class". Essa desproporcionalidade fica mais clara quando a visualizamos no gráfico de barras. </div> </i>

In [ ]:
# Contagem da variável Class
table(dados$Class)

In [ ]:
# Gráfico de barras da váriável Class
dados %>%
  ggplot(aes(factor(Class), fill = factor(Class))) +
  geom_bar()

## 4. Construindo modelo

<i> Nós temos um problema de classificação quando desejamos estimar o número de transações fraudulentas com cartão de crédito, uma vez que determinada transação pode ser categorizada como (1) fraude e (0) legal. Deste modo, usaremos os seguintes algorítimos: Logistic Regression e Decision Tree. </i>


In [ ]:
# Divisão entre treinamento e teste

set.seed(123)
divisao<- sample.split(dados$Class, SplitRatio = 0.7)
treinamento<- subset(dados, divisao == T)
teste<- subset(dados, divisao == F)

In [ ]:
# Visualizando a dimensão
dim(treinamento)
dim(teste)

In [ ]:
# Verificação do processo de randomização
prop.table(table(treinamento$Class))
prop.table(table(teste$Class))


### 4.1 Regressão Logística

In [ ]:
# Modelo glm

model_glm<- glm(Class ~ ., treinamento, family = binomial(link = "logit"))

In [ ]:
# Predição do modelo
predict_glm<- predict(model_glm, teste[,-31], type = "response")


In [ ]:
# Visualização do desempenho (matriz de confusão)

previsoes<- ifelse(predict_glm > 0.5, 1, 0)
matriz_confusao<- table(teste[,31], previsoes)
confusionMatrix(matriz_confusao, positive = "1")


In [ ]:
# Pseudo R ao quadrado (ajuste geral do modelo)

RsqGLM(model_glm)

In [ ]:
# Gráfico com acurácia geral do modelo (curva ROC)

ROCRpred<- prediction(predict_glm, teste$Class)
performance<- performance(ROCRpred, "tpr", "fpr")
plot(performance, colorize=TRUE)

In [ ]:
# Gráfico curva AUC com intervalo de confiança

curva_auc <- roc(teste$Class,predict_glm,
                smoothed = TRUE,
                ci=TRUE, ci.alpha=0.9, stratified=FALSE,
                plot=TRUE, auc.polygon=TRUE, max.auc.polygon=TRUE, grid=TRUE,
                print.auc=TRUE, show.thres=TRUE)


ci <- ci.se(curva_auc)
plot(ci, type="shape", col="lightblue")

In [ ]:
# Gráficos dos aspectos do modelo
aspectos_modelo <- evalmod(scores = predict_glm, labels = teste$Class, mode="basic")
autoplot(aspectos_modelo)

### 4.2 Árvore de decisão

In [ ]:
# Modelo Decision Tree
model_tree<- rpart(Class ~ ., data = treinamento, method = "class", minbucket = 10, maxdepth = 5)

In [ ]:
# Plotar a árvore
prp(model_tree, extra = 106) 

In [ ]:
# Predição do modelo
predict_tree<- predict(model_tree, teste[,-31], type = "class")

In [ ]:
# Tabela de comparação
CrossTable(teste$Class, predict_tree, prop.chisq = FALSE, 
           prop.c = FALSE, prop.r = FALSE, dnn = c("Actual", "Predicted"))

In [ ]:
# Matriz de confusão
confusionMatrix(predict_tree, teste$Class, positive = "1")

### 4.3 Sumário dos modelos

<p></p>
Logistic Regression:
<p></p>
<li>Accuracy 99.93%</li>
<li>Sensitivity 91.17%</li>
<li>Specificity 99.93%</li>
<p></p>
DecisionTrees:
    <p></p>
<li>Accuracy 99.94%</li>
<li>Sensitivity 81.08%</li>
<li>Specificity 99.97%</li>
 <p></p>
 <p>
*Sensitivity -> positivos corretamente identificados<br />  
*Specificity -> negativos corretamente identificados</p>

<p><i>
Diego Ramirez<br />
daarfarias@hotmail.com<br />
https://github.com/daarfarias<br />
    
</i></p>